In [1]:
!pip install camelot

  Using cached Camelot-12.06.29.tar.gz (3.5 MB)
  Preparing metadata (setup.py) ... done
  Using cached SQLAlchemy-0.7.10.tar.gz (3.5 MB)
  Preparing metadata (setup.py) ... done
  Using cached Elixir-0.7.1.tar.gz (47 kB)
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for sqlalchemy-migrate>=0.7.1 from https://files.pythonhosted.org/packages/1a/ba/0725bf5f54d0c5ed7c1da025215f288fb68ef4cc4966abfb76c1926ee0af/sqlalchemy_migrate-0.13.0-py2.py3-none-any.whl.metadata
  Using cached sqlalchemy_migrate-0.13.0-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached xlwt-0.7.2.zip (131 kB)
  Preparing metadata (setup.py) ... done
  Using cached xlrd-0.7.1.zip (125 kB)
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for pbr>=1.8 from https://files.pythonhosted.org/packages/1d/44/6a65ecd630393d47ad3e7d5354768cb7f9a10b3a0eb2cd8c6f52b28211ee/pbr-6.1.0-py2.py3-none-any.whl.metadata
  Using cached pbr-6.1.0-py2.py3-none-any.whl.metadata (3.4 kB)


In [2]:
%pip install langchain
%pip install -U langchain-community
%pip install python-docx 
%pip install chromadb
%pip install sentence-transformers
%pip install camelot-py 
%pip install tabula-py 
%pip install python-docx 
%pip install ghostscript
%pip install camelot



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached Camelot-12.06.29.tar.gz (3.5 MB)
  Preparing metadata (setup.py) ... done
  Using cached SQLAlchemy-0.7.10.tar.gz (3.5 MB)
  Preparing metadata (setup.py) ... done
  Using cached Elixir-0.7.1.tar.gz (47 kB)
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for sqlalchemy-migrate>=0.7.1 from https://files.pythonhosted.org/packages/1a/ba/0725bf5f54d0c5ed7c1da025215

In [3]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import requests
import re


In [4]:
import torch
import os
from docx import Document


In [5]:
import camelot
import tabula
from docx import Document
from tqdm import tqdm
import pandas as pd
import time
# import pythoncom
# from win32com.client import Dispatch
import requests

/Users/ivanlopatkin/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/ivanlopatkin/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [6]:
def clean_text(text):
    """
    Функция для удаления "мусора" из текста, такого как пустые строки, таблицы,
    лишние пробелы, номера пунктов и лишние символы.
    """
    # Удаление лишних символов табуляции, переносов строк и последовательностей пробелов
    cleaned_text = re.sub(r'\n+', '\n', text)  # Удаление повторяющихся переносов строк
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Замена множественных пробелов на один пробел

    # Удаление строк, содержащих "Таблица" и другие нерелевантные технические данные
    cleaned_text = re.sub(r'(Таблица\s*\d+\.\d+|Приложение\s*\d+)', '', cleaned_text)
    
    # Удаление номеров пунктов и заголовков с номерами (например, 2.3.1, 4 ХАРАКТЕРИСТИКИ ИЗАВ)
    cleaned_text = re.sub(r'(\d+\.\d+(\.\d+)?|^\d+[\sА-Яа-яЁё ]+)', '', cleaned_text)

    # Удаление оставшихся технических фрагментов
    cleaned_text = re.sub(r'[^\w\s,.()ёЁА-Яа-я-]', '', cleaned_text)  # Удаление спецсимволов, кроме пунктуации
    
    # Удаление лишних пробелов, возникающих после чистки
    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text).strip()
    
    return cleaned_text



In [7]:
def load_and_split_documents(documents_folder):
    """
    Загрузка документов из локальной директории и разбиение их на фрагменты.
    """
    # Загрузка документов с указанием кодировки utf-8
    loader = DirectoryLoader(
        documents_folder, glob="*.txt", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"}
    )
    documents = loader.load()
    
    for i in range(len(documents)):
        documents[i].page_content = clean_text(documents[i].page_content)

    # Разбиение документов на фрагменты
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    texts = text_splitter.split_documents(documents)
    return texts

In [8]:
def create_vectorstore(texts, embedding_model_name):
    """
    Создание эмбеддингов и сохранение их в векторном хранилище ChromaDB.
    """
    # Создание эмбеддингов
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
    
    # Создание векторного хранилища ChromaDB
    vectorstore = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        persist_directory="chroma_db"
    )
    vectorstore.persist()
    return vectorstore

### Table Summarization part

### Yandex GPT

In [9]:
def call_api(system_prompt: str, prompt: str):
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion" 
    
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Api-Key AQVNwy5Z1_keoKl1F1R6BxAXLOKCiq5h1NEzd4Dw", # апи ключ
        "x-folder-id": "b1ghupvijk4nvsbqtcv3" # id каталога
    }
    
    payload = {
        "modelUri": "gpt://b1ghupvijk4nvsbqtcv3/yandexgpt-32k/rc", # модель https://yandex.cloud/ru/docs/foundation-models/concepts/yandexgpt/models

        # опции
        "completionOptions": {
            "stream": False,
            "temperature": 0.1,
            "maxTokens": "2000"
        },
        
        "messages": [
            {
                "role": "system",
                "text": system_prompt
            },
            {
                "role": "user",
                "text": prompt
            }
        ]
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        result = response.json()
        return result['result']['alternatives'][0]['message']['text']
    else:
        raise Exception(f"API request failed with status code {response.status_code}: {response.text}")

In [10]:
def summarize_table_with_yandex(df, title):
    table_text = df.to_string()
    system_prompt = """Ты опытный аналитик данных и можешь создавать качественные текстовые описания таблиц. Твоя задача — представить ключевые показатели, выявить основные тренды и сделать краткие, но точные выводы. Если таблица пустая или содержит мало данных, ничего не выводи. Старайся делать описание логичным и структурированным, добавляя такие элементы, как:
                - Основные метрики (средние значения, медиана, стандартное отклонение),
                - Важные тенденции (например, увеличение или уменьшение значений по столбцам или строкам),
                - Выявление выбросов и экстремальных значений,
                - Примеры или сравнительные выводы между значениями.
                Если данные относятся к конкретным категориям (например, по времени, регионам, продуктам), выделяй это в описании."""
    user_prompt = f"Таблица: {title}. {table_text} Создай качественное текстовое описание для данных из таблицы."

    return call_api(system_prompt, user_prompt)

### Экстракт только табличек

In [11]:
def extract_and_summarize_tables(word_path, output_path="final_summary.txt"):
    # doc = set_web_layout(word_path)
    doc = Document(word_path)
    summaries = []
    paragraphs = doc.paragraphs

    for i, table in tqdm(enumerate(doc.tables)):
        title = "Без названия"
        for j, para in enumerate(paragraphs):
            if para._element == table._element.getprevious():
                title = paragraphs[j - 1].text if j > 0 else title
                break

        data = [[cell.text for cell in row.cells] for row in table.rows]
        df = pd.DataFrame(data[1:], columns=data[0])

        summary = summarize_table_with_yandex(df, title)
        time.sleep(1)
        summaries.append(f"Название таблицы: {title}\n{summary}")

    with open(output_path, "w", encoding="utf-8") as file:
        file.write("\n---------------------------------\n".join(summaries))

### Экстракт текста и табличек одновременно

In [14]:
def extract_and_summarize_docx(word_path, output_path="final_summary.txt"):
    doc = Document(word_path)
    summaries = []
    paragraphs = doc.paragraphs
    para_iter = iter(paragraphs) 
    
    for table in tqdm(doc.tables, desc="Processing tables"):
        text_before_table = ""
        for para in para_iter:
            if para._element == table._element.getprevious():
                break
            text_before_table += para.text + "\n"

        title = "Без названия"
        previous_paragraph = table._element.getprevious()
        if previous_paragraph is not None:
            title = previous_paragraph.text

        data = [[cell.text for cell in row.cells] for row in table.rows]
        df = pd.DataFrame(data[1:], columns=data[0])

        summary = summarize_table_with_yandex(df, title)
        time.sleep(1)
        
        summaries.append(text_before_table)
        summaries.append(f"Название таблицы: {title}\n{summary}\n")

    remaining_text = "".join([para.text + "\n" for para in para_iter])
    summaries.append(remaining_text)

    # with open(output_path, "w", encoding="utf-8") as file:
    #     file.write("\n---------------------------------\n".join(summaries))
    
    return "\n---------------------------------\n".join(summaries)

In [16]:
word_path = ... # введи путь к тестовому файлу ../documents/...docx

In [ ]:
dirty_text = extract_and_summarize_docx(word_path) # если нужно сохранить файл - закрой комменты в функции выше

clean_text = clean_text(dirty_text) # препроцессинг всего того, что 

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# Загрузка модели и токенизатора
tokenizer = AutoTokenizer.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")
model = AutoModelForQuestionAnswering.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")

# Инициализация пайплайна
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

def run_query(retriever, query):
    """
    Запуск поиска и генерация ответа с использованием модели xlm-roberta.
    """
    # Получение релевантных документов
    docs = retriever.get_relevant_documents(query)
    
    # Формирование контекста из документов
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # Если контекст пустой, возвращаем сообщение об ошибке
    if not context:
        return {
            "result": "Не удалось найти релевантный контекст для ответа.",
            "source_documents": docs
        }
    
    # Использование модели для получения ответа
    qa_input = {
        'question': query,
        'context': context
    }
#     print(qa_input)
    # Вызов модели для генерации ответа
    response = qa_pipeline(qa_input)
    
    # Возврат результата и исходных документов
    result = {
        "result": response['answer'],
        "source_documents": docs
    }
    
    return result


In [ ]:
def convert_docx_to_txt(folder_path):
    # Проверяем, существует ли папка
    if not os.path.isdir(folder_path):
        print(f"Папка {folder_path} не найдена.")
        return

    # Проходим по всем файлам в указанной папке
    for filename in os.listdir(folder_path):
        if filename.endswith('.docx'):  # Проверяем, что это .docx файл
            docx_path = os.path.join(folder_path, filename)
            txt_path = os.path.join(folder_path, filename.replace('.docx', '.txt'))

            # Чтение содержимого .docx файла
            doc = Document(docx_path)
            text = ""
            for paragraph in doc.paragraphs:
                text += paragraph.text + "\n"

            # Запись содержимого в .txt файл
            with open(txt_path, 'w', encoding='utf-8') as txt_file:
                txt_file.write(text)

            print(f"Файл {filename} успешно преобразован в {txt_path}")

# Пример использования


In [11]:
documents_folder = "/kaggle/input/ecology-hack"  # Путь к папке с документами
query = "Какие существуют типы источников выбросов? Привести примеры каждого источника."  # Пользовательский запрос

In [12]:
# Загрузка и разбиение документов
texts = load_and_split_documents(documents_folder)

In [13]:
texts[:2], len(texts)

([Document(metadata={'source': '/kaggle/input/ecology-hack/work_train_book_2.txt'}, page_content='Название таблицы Расчет рассеивания для Производственной площадки карьер I проведен на лето. Данная таблица представляет собой расчет рассеивания для Производственной площадки карьер I, проведенный на лето. В таблице представлены данные о расчетных площадках и точках, включая координаты, типы точек и комментарии. Основные метрики 1. Координаты середины 1-й стороны (м) - Среднее значение по X -299,60 м - Среднее значение по Y 80,20 м 2. Координаты середины 2-й стороны (м) - Среднее значение по X 400,40 м - Среднее значение по Y 80,20 м 3. Ширина (м) - Среднее значение 600,00 м 4. Зона влияния (м) - Среднее значение по ширине 50,00 м - Среднее значение по длине 50,00 м 5. Высота (м) - Среднее значение 2,00 м Важные тенденции - Координаты середины 1-й стороны (м) - Координаты X варьируются от -299,60 до 271,33 м. - Координаты Y варьируются от 3,79 до 260,38 м. - Координаты середины 2-й сторон

In [14]:
len(texts[0].page_content)

997

In [15]:
'cointegrated/rubert-tiny2'

'cointegrated/rubert-tiny2'

In [16]:
# Создание векторного хранилища
vectorstore = create_vectorstore(texts, 'sergeyzh/LaBSE-ru-turbo')

/tmp/ipykernel_30/311526257.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)


modules.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/56.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/887 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/576k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/196 [00:00<?, ?B/s]

/tmp/ipykernel_30/311526257.py:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [17]:
vectorstore

In [23]:
retriever = vectorstore.as_retriever()
docs = retriever.get_relevant_documents(query)
    
# Формирование контекста из документов
context = "\n\n".join([doc.page_content for doc in docs])

/tmp/ipykernel_30/3384211756.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


In [27]:
prompt = f"""Ты — эксперт в области экологии, защиты окружающей среды и экологического права. Твоя задача — помогать пользователям находить ответы на вопросы об экологии, экологических нормах, правилах и стандартах.

Контекст: {context}

Вопрос: {query}

Оцени, подходит ли данный контекст для ответа на вопрос. Если контекст является достаточным для ответа, используй его. В любом случае, предоставь максимально подробный и точный ответ на вопрос, используя всю свою экспертную информацию.
"""

In [29]:
len(prompt)

4011

In [21]:
# Настройка ретривера
retriever = vectorstore.as_retriever()

# Запуск запроса
result = run_query(retriever, query)

# Вывод ответа и исходных документов
print("Ответ:")
print(result["result"])
print("\nИсходные документы:")
# for doc in result["source_documents"]:
#     print(doc.metadata["source"])


/tmp/ipykernel_30/1530947629.py:15: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


{'question': 'Какие существуют типы источников выбросов? Привести примеры каждого источника.', 'context': 'с законодательством Российской Федерации в области охраны окружающей среды (технологическим нормативам выбросов, предельно допустимым выбросам, временно согласованным выбросам или временно разрешенным выбросам). 4 ХАРАКТЕРИСТИКИ ИЗАВ, ПОКАЗАТЕЛИ РАБОТЫ ГОУ, СУММАРНЫЕ ВЫБРОСЫ ПО ОНВ Источники выделения загрязняющих веществ Источники выбросов загрязняющих веществ Источники выбросов загрязняющих веществ, имеющие произвольную форму Источники выбросов загрязняющих веществ, ось устья которых отклоняется от вертикали Источники выбросов загрязняющих веществ в случае факельного горения Результаты обследования установок очистки газа и условий их эксплуатации Суммарные выбросы ЗВ в атмосферный воздух, их очистка и утилизация (в целом по ОНВ), тгод 5 РЕЗУЛЬТАТЫ ОПРЕДЕЛЕНИЯ ВЫБРОСОВ ЗАГРЯЗНЯЮЩИХ ВЕЩЕСТВ РАСЧЕТНЫМИ МЕТОДАМИ Расчеты выбросов загрязняющих веществ от источников загрязнения предста

# Получение сабмишена

In [18]:
!pip install openpyxl
!pip install -q evaluate bert_score
!pip install --upgrade nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.


In [19]:
import numpy as np
import pandas as pd

In [20]:
df = pd.read_excel("/kaggle/input/train-dset/train_dset.xlsx", index_col=0)

df.head(5)

,Вопрос,Ответ,Документ
№ п/п,,,
1,Какие существуют типы источников выбросов? При...,"Организованные (дымовые трубы, выходы вентиляц...",Нет
2,Расписать состав тома инвентаризации,"ВВЕДЕНИЕ\n1.СВЕДЕНИЯ О ХОЗЯЙСТВУЮЩЕМ СУБЪЕКТЕ,...",Нет
3,Для каких предприятий разрабатывается инвентар...,Инвентаризация стационарных источников и выбро...,Нет
4,Как присваиваются номера ИВ?,Для каждого ИЗАВ начиная с 01.,Нет
5,Какие бывают методы определения качественных и...,Инструментальные (лабораторные) и расчетные (н...,Нет


In [21]:
import evaluate
from evaluate import load
import nltk

In [22]:
meteor = evaluate.load('meteor')
bertscore = load("bertscore")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [23]:
!pip install mistralai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 5.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.9.0.post0
    Uninstalling python-dateutil-2.9.0.post0:
      Successfully uninstalled python-dateutil-2.9.0.post0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 17.0.0 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22

In [24]:
from mistralai import Mistral

In [25]:
api_key = '1MbVc4n2iUxhJgOx1lfaUXuXJtzOVG0b'

model = "mistral-large-latest"
client = Mistral(api_key=api_key)



In [26]:
len(df)

84

In [27]:
answers = []

In [ ]:
# for i in range(len(df)):
#     query = df.iloc[i]['Вопрос']
    
#     retriever = vectorstore.as_retriever()
#     docs = retriever.get_relevant_documents(query)
#     # Формирование контекста из документов
#     context = "\n".join([doc.page_content for doc in docs])
    
#     prompt = f"""Контекст: {context}
#     Вопрос: {query}
#     Оцени, подходит ли данный контекст для ответа на вопрос! Если контекст является достаточным для ответа, используй его. В любом случае, предоставь максимально подробный и точный ответ на вопрос, используя всю свою экспертную информацию.
#     """
    
#     chat_response = client.chat.complete(
#     model=model,
#     messages=[
#         {"role": "system", "content":"Ты — эксперт в области экологии, защиты окружающей среды и экологического права. Твоя задача — помогать пользователям находить ответы на вопросы об экологии, экологических нормах, правилах и стандартах!"},
#         {"role": "user", "content": prompt}
#     ]
#     )
#     answers.append(chat_response)
#     print(i+1)

/tmp/ipykernel_30/1122691930.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


1
2
3
4


In [ ]:
answers[:3], len(answers)

In [ ]:
answers_text = [answers[i].choices[0].message.content for i in range(len(answers))]
answers_text[:3], len(answers_text)

In [ ]:
df['Ответ_модели'] = answers_text

In [ ]:
df.head()

In [ ]:
score_meteor = meteor.compute(predictions=df['Вопрос'].values.tolist(), references=df['Ответ_модели'].values.tolist())
print (score_meteor)

In [ ]:
score_bert = bertscore.compute(predictions=df['Вопрос'].values.tolist(), references=df['Ответ_модели'].values.tolist(), lang="ru")

In [ ]:
print ('Presicion', np.array(score_bert['precision']).mean())
print ('Recall', np.array(score_bert['recall']).mean())
print ('F1', np.array(score_bert['f1']).mean())